In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [38]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [39]:
url = "https://www.transtats.bts.gov/AverageFare/"
browser.visit(url)

In [40]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [41]:
def scrape_page(html, ls, yr, qtr):
    table = soup.find('table', id='GridView1')
    t_rows = table.find_all('tr')
    for row in t_rows:
        dict = scrape_row(row)
        if dict is not None:
            dict['year'] = yr
            dict['quarter'] = qtr
            ls.append(dict)
    return table

def scrape_row(table_row):
    try:
        tds = table_row.find_all('td')
        state = tds[3].text
        if state == 'HI':
            airport = tds[1].text
            city = tds[2].text
            avg_fare = float(tds[4].text.replace(",", ""))
            adjusted_fare = float(tds[5].text.replace(",", ""))
            dict = {
                "airport" : airport,
                "city" : city,
                "Average Fare": avg_fare,
                "Inflation Adjusted Average Fare": adjusted_fare
            }
            return dict
        return None
        
    except:
        pass


In [42]:
years = [i for i in range(1993, 2022)]
quarters = [1, 2, 3, 4]

In [43]:
import pandas as pd
import time

In [44]:
ls = []
for yr in years:
    browser.find_by_id("dlstYear").select(yr)
    for qtr in quarters:
        browser.find_by_id("dlstQuarter").select(qtr)
        browser.find_by_id("btnSubmit").click()
        time.sleep(0.5)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        scrape_page(html, ls, yr, qtr)
df = pd.DataFrame(ls)
df.head()

,airport,city,Average Fare,Inflation Adjusted Average Fare,year,quarter
0,HNL,Honolulu,103.28,211.15,1993,1
1,OGG,Kahului,48.83,99.84,1993,1
2,KOA,Kona,45.96,93.98,1993,1
3,LIH,Lihue,48.21,98.56,1993,1
4,ITO,Hilo,42.72,87.34,1993,1


In [45]:
df.to_csv("airfare_1993_2021.csv", index=False)